# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,buala,-8.1450,159.5921,299.98,83,95,2.41,SB,1729976721
1,1,haiku-pauwela,20.9219,-156.3051,299.40,83,75,10.73,US,1729976722
2,2,byelaazyorsk,52.4731,25.1784,279.95,75,7,2.17,BY,1729976724
3,3,iqaluit,63.7506,-68.5145,272.00,80,75,3.60,CA,1729976726
4,4,waitangi,-43.9535,-176.5597,288.05,98,100,4.47,NZ,1729976729


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
# Configure the map plot using hvplot
city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="Humidity",
    frame_width=800,
    frame_height=600,
    title="City Humidity Map"
)    # YOUR CODE HERE

# Display the map
 # YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:

# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 25) |
    (city_data_df["Humidity"] < 60) |
    (city_data_df["Cloudiness"] < 20)
].dropna()

# Display sample data
ideal_cities_df.head()
# YOUR CODE HERE

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,byelaazyorsk,52.4731,25.1784,279.95,75,7,2.17,BY,1729976724
5,5,pendleton,45.6721,-118.7886,298.38,32,0,3.09,US,1729976730
7,7,bethel,41.3712,-73.4140,288.92,41,40,4.12,US,1729976732
11,11,sirjan,29.4520,55.6814,291.30,41,59,2.36,IR,1729976737
14,14,aoulef,26.9667,1.0833,299.04,27,0,2.01,DZ,1729976741


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,byelaazyorsk,BY,52.4731,25.1784,75,
5,pendleton,US,45.6721,-118.7886,32,
7,bethel,US,41.3712,-73.4140,41,
11,sirjan,IR,29.4520,55.6814,41,
14,aoulef,DZ,26.9667,1.0833,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius =10000  # meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
byelaazyorsk - nearest hotel: Аздараўленчы цэнтр «Энергія»
pendleton - nearest hotel: Howard Johnson Pendleton
bethel - nearest hotel: Hampton Inn Danbury
sirjan - nearest hotel: خانه معلم سیرجان
aoulef - nearest hotel: دار الضياف
greenville - nearest hotel: Quality Inn
enoch - nearest hotel: Travelodge Cedar City
carnarvon - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
korla - nearest hotel: Silver Star Hotel
hamilton - nearest hotel: North Vista Manor
constantia - nearest hotel: Hotel Dali
qaqortoq - nearest hotel: Hotel Qaqortoq
puno - nearest hotel: Huchuy Wasi
riberalta - nearest hotel: Hotel Jomali
albany - nearest hotel: No hotel found
kolyshley - nearest hotel: No hotel found
marshall - nearest hotel: Wyndham Garden Lake-Guntersville
ipojuca - nearest hotel: Pousada da Branca
alta - nearest hotel: Scandic Alta
thompson - nearest hotel: Thompson Inn
mahoba - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
crane - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
2,byelaazyorsk,BY,52.4731,25.1784,75,Аздараўленчы цэнтр «Энергія»
5,pendleton,US,45.6721,-118.7886,32,Howard Johnson Pendleton
7,bethel,US,41.3712,-73.4140,41,Hampton Inn Danbury
11,sirjan,IR,29.4520,55.6814,41,خانه معلم سیرجان
14,aoulef,DZ,26.9667,1.0833,27,دار الضياف


In [23]:
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,byelaazyorsk,BY,52.4731,25.1784,75,Аздараўленчы цэнтр «Энергія»
5,pendleton,US,45.6721,-118.7886,32,Howard Johnson Pendleton
7,bethel,US,41.3712,-73.4140,41,Hampton Inn Danbury
11,sirjan,IR,29.4520,55.6814,41,خانه معلم سیرجان
14,aoulef,DZ,26.9667,1.0833,27,دار الضياف


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot to include hotel name and country in hover
hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    frame_width=800,
    frame_height=600,
    title="Ideal Weather Cities and Nearby Hotels"
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)